# TSA Chapter 3: Rolling ARIMA Forecast Evaluation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch3/TSA_ch3_rolling_forecast/TSA_ch3_rolling_forecast.ipynb)

This notebook demonstrates:
- Out-of-sample rolling 1-step forecast with expanding window
- MAE and RMSE accuracy metrics.


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
ar = np.array([1, -0.7])
ma = np.array([1, 0.4])
stationary = ArmaProcess(ar, ma).generate_sample(nsample=250)
data = np.cumsum(stationary)

train_size = 200
forecasts = []
actuals = []

for t in range(train_size, len(data)):
    model = ARIMA(data[:t], order=(1, 1, 1)).fit()
    fc = model.forecast(steps=1)
    forecasts.append(fc.iloc[0] if hasattr(fc, 'iloc') else fc[0])
    actuals.append(data[t])

forecasts = np.array(forecasts)
actuals = np.array(actuals)
errors = actuals - forecasts

mae = np.mean(np.abs(errors))
rmse = np.sqrt(np.mean(errors**2))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].plot(range(train_size, len(data)), actuals, color='#1A3A6E', linewidth=1.5, label='Actual')
axes[0].plot(range(train_size, len(data)), forecasts, color='#DC3545', linewidth=1.5,
             linestyle='--', label='1-step Forecast')
axes[0].axvline(x=train_size, color='gray', linestyle=':', alpha=0.5)
axes[0].set_title('Rolling 1-Step Forecast', fontweight='bold')
axes[0].set_xlabel('Time')
axes[0].legend()

axes[1].plot(errors, color='#1A3A6E', linewidth=0.8)
axes[1].axhline(0, color='red', linestyle='--')
axes[1].set_title(f'Forecast Errors (MAE={mae:.3f}, RMSE={rmse:.3f})', fontweight='bold')
axes[1].set_xlabel('Forecast Step')
axes[1].set_ylabel('Error')

plt.tight_layout()
save_chart(fig, 'ch3_rolling_forecast')
plt.show()